### Step 0: specify the Free-form API urls


In [26]:
#Free form SDS API url
url='https://synthetic-data-generator-oaqw25.ai-workbench.eng-ml-l.vnu8-sqze.cloudera.site/synthesis/freeform' 
#Free form eval SDS API url
url_eval = 'https://synthetic-data-generator-oaqw25.ai-workbench.eng-ml-l.vnu8-sqze.cloudera.site/synthesis/evaluate_freeform'


### Step 1: Real Data Reference
**Purpose**: Load sample real-world lending data to understand structure for synthetic generation  
**Key Features**:  
- Contains 27 fields including loan amount, interest rate, employment details, and credit history  
- Example fields: `loan_amnt` (loan amount), `int_rate` (interest rate), `emp_title` (job title)  
- Used as reference for generating realistic synthetic data patterns


In [14]:
#These are the lending datasets and how they look like
import pandas as pd
OutputFile='lending_club_loan_two.csv'
data = pd.read_csv(OutputFile)
data

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,10000.0,36 months,11.44,329.48,B,B4,Marketing,10+ years,RENT,117000.0,...,16.0,0.0,36369.0,41.8,25.0,w,INDIVIDUAL,0.0,0.0,"0174 Michelle Gateway\r\nMendozaberg, OK 22690"
1,8000.0,36 months,11.99,265.68,B,B5,Credit analyst,4 years,MORTGAGE,65000.0,...,17.0,0.0,20131.0,53.3,27.0,f,INDIVIDUAL,3.0,0.0,"1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113"
2,15600.0,36 months,10.49,506.97,B,B3,Statistician,< 1 year,RENT,43057.0,...,13.0,0.0,11987.0,92.2,26.0,f,INDIVIDUAL,0.0,0.0,"87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113"
3,7200.0,36 months,6.49,220.65,A,A2,Client Advocate,6 years,RENT,54000.0,...,6.0,0.0,5472.0,21.5,13.0,f,INDIVIDUAL,0.0,0.0,"823 Reid Ford\r\nDelacruzside, MA 00813"
4,24375.0,60 months,17.27,609.33,C,C5,Destiny Management Inc.,9 years,MORTGAGE,55000.0,...,13.0,0.0,24584.0,69.8,43.0,f,INDIVIDUAL,1.0,0.0,"679 Luna Roads\r\nGreggshire, VA 11650"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396025,10000.0,60 months,10.99,217.38,B,B4,licensed bankere,2 years,RENT,40000.0,...,6.0,0.0,1990.0,34.3,23.0,w,INDIVIDUAL,0.0,0.0,"12951 Williams Crossing\r\nJohnnyville, DC 30723"
396026,21000.0,36 months,12.29,700.42,C,C1,Agent,5 years,MORTGAGE,110000.0,...,6.0,0.0,43263.0,95.7,8.0,f,INDIVIDUAL,1.0,0.0,"0114 Fowler Field Suite 028\r\nRachelborough, ..."
396027,5000.0,36 months,9.99,161.32,B,B1,City Carrier,10+ years,RENT,56500.0,...,15.0,0.0,32704.0,66.9,23.0,f,INDIVIDUAL,0.0,0.0,"953 Matthew Points Suite 414\r\nReedfort, NY 7..."
396028,21000.0,60 months,15.31,503.02,C,C2,"Gracon Services, Inc",10+ years,MORTGAGE,64000.0,...,9.0,0.0,15704.0,53.8,20.0,f,INDIVIDUAL,5.0,0.0,"7843 Blake Freeway Apt. 229\r\nNew Michael, FL..."


### Step 2a: Synthetic Data Generation
**Key Parameters**:
- **Model**: Meta Llama-3.1-70B (large language model running on CAII)  
- **Temperature 1.0**: High creativity for diverse outputs
- **max_tokens 8192**: Enough tokens to generate each output fully.
- **Examples**: Uses `ExamplesLoanData.json` for pattern learning
  -  Fields need to match fields in the prompt in the same sequence.
  -  Reduct PII
  -  Ensure diversity of examples
  -  Consistency in categories: Ensure fields like `home_ownership` use predefined values (RENT, OWN, MORTGAGE, OTHER).  
- **Prompt Structure**:  
  - Specifies all 27 fields with precise definitions  
  - Requires logical relationships between variables (e.g., interest rates matching credit risk)
  - Explain each field clearly in the expected sequence.


In [27]:
%%time
import requests
import os

# Get API key from environment variable if within CDSW app/session
api_key = os.environ.get('CDSW_APIV2_KEY')


# URL for synthesis


# Add the API key to headers with proper Authorization format
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json'
}

# If API key exists, add it to the headers
if api_key:
    headers['Authorization'] = f'Bearer {api_key}'
else:
    print("Warning: No API key provided")

# Payload for data synthesis
payload = {
  "inference_type": "CAII",
  "caii_endpoint": "https://caii-prod-long-running.eng-ml-l.vnu8-sqze.cloudera.site/namespaces/serving-default/endpoints/llama-31-70b-instruct-8xl40s/v1/chat/completions",
  "model_id": "meta/llama-3.1-70b-instruct",

  "is_demo": False,
  "num_questions": 30,
  "custom_prompt": """
 You need to create profile data for the LendingClub company which specialises in lending various types of loans to urban customers.
 

You need to generate the data in the same order for the following  fields (description of each field is followed after the colon):

loan_amnt: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.
term: The number of payments on the loan. Values are in months and can be either 36 months or 60 months.
int_rate: Interest Rate on the loan
installment: The monthly payment owed by the borrower if the loan originates.
grade: LC assigned loan grade (Possible values: A, B, C, D, E, F, G)
sub_grade: LC assigned loan subgrade (Possible sub-values: 1-5 i.e A5)
emp_title: The job title supplied by the Borrower when applying for the loan.
emp_length: Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.
home_ownership: The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER
annual_inc: The self-reported annual income provided by the borrower during registration.
verification_status: Indicates if income was verified by LC, not verified, or if the income source was verified
issue_d: The month which the loan was funded
loan_status: Current status of the loan
purpose: A category provided by the borrower for the loan request.
title: The loan title provided by the borrower
dti: A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.
earliest_cr_line: The month the borrower's earliest reported credit line was opened
open_acc: The number of open credit lines in the borrower's credit file.
pub_rec: Number of derogatory public records
revol_bal: Total credit revolving balance
revol_util: Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.
total_acc: The total number of credit lines currently in the borrower's credit file
initial_list_status: The initial listing status of the loan. Possible values are – W, F
application_type: Indicates whether the loan is an individual application or a joint application with two co-borrowers
mort_acc: Number of mortgage accounts.
pub_rec_bankruptcies: Number of public record bankruptcies
address: The physical address of the person


""",
  "model_params": {
    "temperature": 1.0, # range 0-2 tyically, low temperature gives high accuracy, high temperature gives diversity
    "top_p": 1.0,
    "top_k": 250,       
    "max_tokens": 8192
  },
  "use_case": "custom",
  "topics": [
    "Financial data"
  ],
  "example_path": "ExamplesLoanData.json"
}

# Make the POST request
response = requests.post(url, headers=headers, json=payload)

# Display the response
print(response.status_code)
print(response.json())


200
{'job_name': 'synth_job_3be7', 'job_id': 'zxjk-eg3z-yo6x-8wrp'}
CPU times: user 36.4 ms, sys: 4.04 ms, total: 40.5 ms
Wall time: 2.4 s


### Step 2b: Synthetic Data Inspection
**Sample Output Analysis**:  
- Generated all 27 fields as the original data
- Co-dependent variables vary as expected (i.e. a lower loan grade pays higher interest on the loan)
- Contains plausible occupations (`Software Engineer`, `Teacher`)  
- Addresses follow realistic patterns (`2345 Tech Lane San Jose, CA 95123`)  


In [28]:
import pandas as pd
import json
InputFile='freeform_data_llama_20250415T232752589_final.json'
with open(InputFile,'r') as f:
  data_j = json.load(f)
df=pd.DataFrame(data_j)
df

,Seeds,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,Financial data,14400.0,36 months,9.99,427.03,A,A3,VP Marketing,10+ years,MORTGAGE,...,21.0,0.0,16474.0,55.8,34.0,w,INDIVIDUAL,2.0,0.0,"246 Annette Inlet Suite 171\r\nEast Arielle, M..."
1,Financial data,16000.0,60 months,17.63,354.99,D,D4,Lecturer,5 years,RENT,...,19.0,0.0,21926.0,70.6,32.0,f,JOINT,1.0,0.0,3049 Guillermo Parks Apt. 344\r\nWest Rodgerha...
2,Financial data,10000.0,36 months,13.39,349.11,E,E1,Sales Associate,6 years,MORTGAGE,...,12.0,0.0,9994.0,80.2,26.0,f,INDIVIDUAL,2.0,0.0,"512 Romero Ways Apt. 559\r\nWest Kirsten, MA 1..."
3,Financial data,16800.0,36 months,7.14,487.47,B,B5,Purchasing Manager,7 years,RENT,...,20.0,0.0,38658.0,96.5,35.0,w,JOINT,0.0,0.0,"43973 Jade Flats Apt. 978\r\nPorterfurt, RI 01722"
4,Financial data,14000.0,60 months,10.99,286.99,B,B1,Law Enforcement Officer,4 years,RENT,...,24.0,0.0,35757.0,73.3,38.0,f,INDIVIDUAL,1.0,0.0,"02457 Kramer Overpass Suite 701\r\nPaucek, VT ..."
5,Financial data,18250.0,60 months,16.99,379.47,D,D2,Oilfield,6 years,RENT,...,14.0,0.0,18715.0,58.8,22.0,w,INDIVIDUAL,0.0,0.0,"816 Emmet Mount Apt. 150\r\nPort Natalia, SD 0..."
6,Financial data,7231.0,36 months,21.71,212.89,E,E3,Financial analyst,3 years,RENT,...,13.0,1.0,7457.0,61.4,23.0,f,INDIVIDUAL,0.0,0.0,"9609 Gates Club Apt. 980\r\nNew Ariyah, LA 02510"
7,Financial data,13450.0,60 months,12.79,274.76,B,B1,Academic advisor,7 years,MORTGAGE,...,16.0,0.0,26154.0,51.3,28.0,f,INDIVIDUAL,1.0,0.0,3994 Beckstead Fork Apt. 928\r\nSouth Caitlynf...
8,Financial data,20000.0,36 months,12.13,651.92,C,C3,Director of sales,5 years,MORTGAGE,...,14.0,0.0,34219.0,79.1,30.0,w,INDIVIDUAL,2.0,0.0,"9176 Christy Mount Apt. 370\r\nReynoldsbury, M..."
9,Financial data,24551.0,60 months,14.31,444.69,B,B2,Laundry worker,9 years,RENT,...,18.0,0.0,24014.0,46.4,28.0,f,INDIVIDUAL,0.0,0.0,"4012 Diane Glen Apt. 633\r\nReecestown, MN 01780"


### Section 3a: LLM-Based Evaluation
**Evaluation Process**:  
1. **Prompt**:
   - Provide all required definitions and checks needed for evaluation.
   - Ensure consistency of definitions with the generation prompt
   - Explain clearly how the LLM will score
   - give the LLM examples on what to check. For example, asking the LLM specifically to check for data co-dependencies, temporal consistency and field validity improves evaluation quality.

2. **Scoring**:
   - Specify a scale (1-5 scale in this example):
   - Penalties for mistakes (e.g. field name mismatches, inconsistent field relationships, etc)


3. **Parameter tuning**: Use low `temperature` (e.g., 0.0) to prioritize accuracy over diversity.  


In [29]:
import requests
import os
#********************Accessing Application**************************
# Get API key from environment variable if withinin CDSW app/session.
# To get your API key for using outside CDSW app/session follow given link.
# https://docs.cloudera.com/machine-learning/cloud/api/topics/ml-api-v2.html
api_key = os.environ.get('CDSW_APIV2_KEY')


# Below is your application API URL, you can look at swagger documentation for all existing # endpoints for current application
# https://<application-subdomain>.<workbench-domain>/docs--> will take user to swagger documentaion
# Link to application can be found on application details page within CAI Workbench.


# URL for evaluation

# Add the API key to headers with proper Authorization format
headers = {
    'accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {api_key}'  # Format as specified in the documentation
}   

# The prompt for evaluation
custom_prompt = """Below is financial data synthesized by an LLM which have the following fields (description of each field is followed after the colon):

loan_amnt: The listed amount of the loan applied for by the borrower. If at some point in time, the credit department reduces the loan amount, then it will be reflected in this value.
term: The number of payments on the loan. Values are in months and can be either 36 or 60.
int_rate: Interest Rate on the loan
installment: The monthly payment owed by the borrower if the loan originates.
grade: LC assigned loan grade (Possible values: A, B, C, D, E, F, G)
sub_grade: LC assigned loan subgrade (Possible sub-values: 1-5 i.e A5)
emp_title: The job title supplied by the Borrower when applying for the loan.
emp_length: Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years.
home_ownership: The home ownership status provided by the borrower during registration or obtained from the credit report. Our values are: RENT, OWN, MORTGAGE, OTHER
annual_inc: The self-reported annual income provided by the borrower during registration.
verification_status: Indicates if income was verified by LC, not verified, or if the income source was verified
issue_d: The month which the loan was funded
loan_status: Current status of the loan
purpose: A category provided by the borrower for the loan request.
title: The loan title provided by the borrower
dti: A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.
earliest_cr_line: The month the borrower's earliest reported credit line was opened
open_acc: The number of open credit lines in the borrower's credit file.
pub_rec: Number of derogatory public records
revol_bal: Total credit revolving balance
revol_util: Revolving line utilization rate, or the amount of credit the borrower is using relative to all available revolving credit.
total_acc: The total number of credit lines currently in the borrower's credit file
initial_list_status: The initial listing status of the loan. Possible values are – W, F
application_type: Indicates whether the loan is an individual application or a joint application with two co-borrowers
mort_acc: Number of mortgage accounts.
pub_rec_bankruptcies: Number of public record bankruptcies
address: The physical address of the person:

Rate the quality of the generated whether they reflect real financial data and the variables make sense when considered to together.

For example, int_rate which represents interest rate should reflect real world interest rates during the issue_d period (The month which the loan was funded) but also reflect the risk profile of the person, dti, earliest_cr_line and other fields.
For example, the installment need to make sense when considering the loan amount, the term and the int_rate.

Make sure you analyze as many condependence between the fields and subtract 1 point for each co-dependency that does not reflect real data.
Subtract 2 points if there is a mistake in the field keys or field values.
Subtract 2 points for each other mistake you see with the data.
Give a score rating 1-5 for the given data. 
"""




# Model parameters
model_params = {
    "temperature": 0.0,
    "top_p": 1.0,
    "top_k": 250,
    "max_tokens": 4096
}

payload = {
    "export_type": "local",
    "display_name": "LendingData",
    "import_path": InputFile,
    "import_type": "local",
    "inference_type": "CAII",
    "caii_endpoint": "https://caii-prod-long-running.eng-ml-l.vnu8-sqze.cloudera.site/namespaces/serving-default/endpoints/llama-31-70b-instruct-8xl40s/v1/chat/completions",
    "model_id": "meta/llama-3.1-70b-instruct",
    "use_case": "custom",
    "is_demo": False,
    "custom_prompt": custom_prompt,
    "model_params": model_params
}

responseEval = requests.post(url_eval, headers=headers, json=payload)

# Print the response
print(responseEval.status_code)
print(responseEval.json())


200
{'job_name': 'LendingData_ca1b', 'job_id': 'ombz-ywdz-2q7k-6a19'}


## Step 3b: Example LLM-as-a-judge output
- Shows the sample in question.  
- Provides a score and a justification for the score which can be used for further filtering.  


In [18]:
import pandas as pd
import json
##############
#Replace this with the output of the LLM-as-a-judge step
LLMJUDGEOUT='row_data_llama_20250415T214813977_evaluated.json'
###############
with open(LLMJUDGEOUT,'r') as f:
  data_j = json.load(f)
print(json.dumps(data_j,indent=4))

{
    "average_score": 4.0,
    "min_score": 4,
    "max_score": 4,
    "evaluated_rows": [
        {
            "row": {
                "Seeds": "Financial data",
                "loan_amnt": 12000.0,
                "term": "36 months",
                "int_rate": 10.44,
                "installment": 382.85,
                "grade": "A",
                "sub_grade": "A2",
                "emp_title": "Engineer",
                "emp_length": "4 years",
                "home_ownership": "OWN",
                "annual_inc": 120000.0,
                "verification_status": "Verified",
                "issue_d": "Mar-2015",
                "loan_status": "Fully Paid",
                "purpose": "small_business",
                "title": "Business loan",
                "dti": 18.53,
                "earliest_cr_line": "Jan-1993",
                "open_acc": 19.0,
                "pub_rec": 0.0,
                "revol_bal": 18797.0,
                "revol_util": 44.5,
                "

## Step 3c: Filtering and Conversion to Tabular Data
- Filter low-quality rows and convert valid data into a DataFrame.  
- Choose a score threshold (e.g., >3.5) to balance quality and quantity based on business needs.  


In [19]:
AllRows=[]
for i in data_j['evaluated_rows']:
  del i['row']['Seeds']
  if i['evaluation']['score'] > 3.5:
      AllRows.append(i['row'])

import pandas as pd
df=pd.DataFrame.from_records(AllRows)
df


,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,...,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,12000.0,36 months,10.44,382.85,A,A2,Engineer,4 years,OWN,120000.0,...,19.0,0.0,18797.0,44.5,30.0,w,INDIVIDUAL,2.0,0.0,"647 Laura Oval Apt. 226\r\nCliburn, CO 88985"
1,9000.0,36 months,10.99,285.29,C,C3,Operations Manager,5 years,MORTGAGE,70000.0,...,14.0,0.0,24176.0,48.1,24.0,w,INDIVIDUAL,1.0,0.0,"8571 Gracie Place\r\nHamilburg, ME 10455"
2,10000.0,36 months,11.49,314.79,B,B3,Program Manager,7 years,MORTGAGE,85000.0,...,15.0,0.0,26286.0,51.3,27.0,f,INDIVIDUAL,1.0,0.0,"7989 Smith Courts Apt. 281\r\nBroadhurst, NM 7..."
3,15000.0,60 months,11.99,283.63,B,B1,Sales,10+ years,RENT,95000.0,...,18.0,0.0,34872.0,63.6,32.0,f,INDIVIDUAL,0.0,0.0,"2896 Alicia Square Apt. 235\r\nBolingbrook, IL..."
4,18000.0,60 months,11.24,332.41,C,C5,Accountant,6 years,RENT,60000.0,...,11.0,0.0,14518.0,61.8,21.0,w,INDIVIDUAL,0.0,0.0,"4348 Timothy Hollow Apt. 495\r\nQuinby, MA 91846"
5,11000.0,60 months,9.99,225.89,A,A1,Sales Manager,5 years,OWN,82000.0,...,19.0,0.0,49458.0,75.2,34.0,f,JOINT,2.0,0.0,"5910 Woods Park\r\nBurkeshire, OK 86416"
6,26000.0,36 months,12.34,823.29,C,C2,Chemical Engineer,2 years,MORTGAGE,124000.0,...,22.0,0.0,63191.0,58.3,41.0,w,INDIVIDUAL,2.0,0.0,"02536 Lizbeth Spur\r\nFayeville, AR 56437"
7,19000.0,60 months,10.19,393.79,B,B2,Data Scientist,3 years,RENT,68000.0,...,18.0,0.0,43701.0,67.4,32.0,w,INDIVIDUAL,1.0,0.0,"44497 Zachariah Mount Apt. 740\r\nTorvmouth, V..."
8,15000.0,36 months,11.74,443.85,D,D4,Marketing Manager,1 year,MORTGAGE,52000.0,...,15.0,0.0,58443.0,69.2,28.0,f,INDIVIDUAL,1.0,0.0,"304 Anderson Field Apt. 567\r\nFleetham, TX 97535"
9,22000.0,36 months,12.29,697.47,C,C4,Software Engineer,3 years,RENT,90000.0,...,18.0,0.0,31480.0,58.7,27.0,f,INDIVIDUAL,0.0,0.0,9612 Roger Street Apt. 145\r\nNew Cristinaberg...


## Step 4: Saving to CSV
- **Export**: Save the cleaned data to a CSV file for downstream use.  
- **Format consistency**: Use a delimiter like `\t` to avoid conflicts with existing data (e.g., commas in addresses).  


In [20]:
OutputFile='data_tab_separated.csv'
df.to_csv(OutputFile, sep='\t')
